In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import nltk
import os
import re
import pickle
import sklearn
import sys
import string

from nltk import TextTilingTokenizer

from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.svm import SVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../../')
sys.path.append(src_dir)

In [3]:
%aimport src.data.delicious_t140
%aimport src.helpers.labels
%aimport src.utils.dataframes, src.utils.clusters, src.utils.metrics, src.utils.distances, src.utils.plotting

In [4]:
from src.features.delicious_t140 import clean_text_delicious
from src.data.delicious_t140 import get_sample_from_cache
from src.helpers.labels import truncate_labels
from src.utils.dataframes import sample_rows
from src.utils.metrics import ranking
from src.utils.clusters import k_medoids
from src.utils.distances import hausdorff
from src.utils.plotting import plot_micro_f1_at_k

from src.helpers.segments import make_distance_matrix_for_segments,vectorize_segments

In [5]:
MODELS_ROOT = os.path.abspath("../../../models/ranking/delicious-mimlsvm/")
DATA_ROOT = "/media/felipe/SAMSUNG/delicious/delicioust140"
INTERIM_DATA_ROOT = os.path.abspath("../../../data/interim/delicious-t140/")

MAX_NB_WORDS = 500
SEED= 42
MIN_TAG_DF=10
SAMPLE_FRAC=10
W=20 # Pseudosentence size (in words) - not specified in the paper, taken from TextTiling default values
K=10 # Size (in sentences) of the block used in the block comparison method - not specified in the paper, taken from TextTiling default values

In [6]:
np.random.seed(SEED)

In [7]:
docs_df = get_sample_from_cache(INTERIM_DATA_ROOT,SAMPLE_FRAC)

In [8]:
docs_df.head(1)

,filename,filetype,hash,tags,url,num_users,num_tags,contents
0,8ef354a6df43362781de531b6176a892.html,html,8ef354a6df43362781de531b6176a892,"jax-ws,techniques,webservices,wsdl,webservice,...",http://netbeans.dzone.com/news/5-techniques-cr...,22,9,5 Techniques for Creating Java Web Services f...


In [9]:
len(docs_df)

14371

In [10]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [11]:
docs_df['sentences'] = docs_df['contents'].map(lambda row: sentence_tokenizer.tokenize(row))

In [12]:
docs_df['num_sentences'] = docs_df['sentences'].map( lambda sents: len(sents))

In [13]:
docs_df.head()

,filename,filetype,hash,tags,url,num_users,num_tags,contents,sentences,num_sentences
0,8ef354a6df43362781de531b6176a892.html,html,8ef354a6df43362781de531b6176a892,"jax-ws,techniques,webservices,wsdl,webservice,...",http://netbeans.dzone.com/news/5-techniques-cr...,22,9,5 Techniques for Creating Java Web Services f...,[ 5 Techniques for Creating Java Web Services ...,72
1,b1ad0a3a0d6a4ed30eea4af6f4d3d666.html,html,b1ad0a3a0d6a4ed30eea4af6f4d3d666,"howto,mda,j2ee,architecture,tutorial,java,temp...",http://www.jaxmag.com/itr/online_artikel/pseco...,11,9,The Enterprise Java Open Source Architecture ...,[ The Enterprise Java Open Source Architecture...,183
2,6426220cf82c1fd5fed5b664f6e95cd5.html,html,6426220cf82c1fd5fed5b664f6e95cd5,"pool,jdbc,opensource,database,library,java,con...",http://xapool.experlog.com/,29,8,'); done = true; } if (NS) { if (WIN) { field...,[ '); done = true; } if (NS) { if (WIN) { fiel...,52
3,9b99818f6a819f2748c84a008b3109d9.html,html,9b99818f6a819f2748c84a008b3109d9,"blogs,transportation,economics,urban,city,sust...",http://thecityfix.com/,19,15,THE CITY FIX: Exploring Sustainable Solutions...,[ THE CITY FIX: Exploring Sustainable Solution...,65
4,9da2b798c0ad44fe5aadfa9274994bd2.html,html,9da2b798c0ad44fe5aadfa9274994bd2,"media,culture",http://www.neural.it/art/2008/05/emotoscope_th...,3,2,"Emotoscope, the emotional time machine - Neur...","[ Emotoscope, the emotional time machine - Neu...",30


In [14]:
docs_df.iloc[0]['sentences'][:1]

[' 5 Techniques for Creating Java Web Services from WSDL | NetBeans Zone @import "/sites/all/files/css/e17f45ffdf706ac58e60d7181c2dab00.css"; @import "/sites/all/themes/dzone/css/extra.css"; \'); //--> Login Join DZone The social network for developers Home Zones.NET Zone Architects Zone Book Zone ColdFusion Zone DZone Eclipse Zone Groovy Zone IT News Javalobby JetBrains Zone NetBeans Zone OSGi Zone PHP Zone Refcardz RIA Zone Ruby Zone Server Zone SOA Zone SQL Zone Web Builder Zone ArticlesAnnouncements How-To Interviews News Opinion/Editorial Reviews A/V MediaPodcasts Presentations Slideshows Videos Downloads Forums Links \'); //--> Home var dzone_title = \'5 Techniques for Creating Java Web Services from WSDL\'; var dzone_url = \'http://netbeans.dzone.com/news/5-techniques-create-web-servic\'; 5 Techniques for Creating Java Web Services from WSDL Submitted by mkuchtiak on Tue, 2008/04/29 - 8:19am.']

In [15]:
tok = TextTilingTokenizer(w=W, k=K)

In [ ]:
def extract_segments(candidates):
    
    try:
        # we must manually insert "\n\n" because this is how 
        # texttilingtokenizer requires candidate boundaries to be 
        # represented.
        segments = tok.tokenize("\n\n".join(candidates))
    except ValueError:
        # this happens when the candidate list is too small for the 
        # text tiling tokenizer to be able to find segments. so just return
        # the original sentences.
        segments= candidates
        
    # now remove the artificially added chars
    segments = [segment.replace("\n\n"," ").strip() for segment in segments]
    
    return segments

In [ ]:
%%time

cache_path = INTERIM_DATA_ROOT+"/docs_df_with_segments-{}-sample-{}.p".format(MAX_NB_WORDS,SEED)

if os.path.isfile(cache_path):
    print('cache hit')
    docs_df = pickle.load(open(cache_path,"rb"))
else:
    docs_df['segments'] = docs_df['sentences'].map(lambda candidates: extract_segments(candidates))
    pickle.dump(docs_df,open(cache_path,"wb"))

In [ ]:
segments = docs_df['segments'].values
documents = docs_df['contents'].values
labels = docs_df["tags"].map(lambda tagstring: tagstring.split(","))
labels = truncate_labels(labels,MIN_TAG_DF)

In [ ]:
def make_train_dataset(distance_matrix, medoid_indices):
    """
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param distance_matrix: MxM matrix with pairwise distances
    :param medoid_indices: array of length N containing the indices of the medoids for each cluster
    :return: distances to medoids (MxN matrix)
    """

    return distance_matrix[:,medoid_indices]

In [ ]:
def make_test_dataset(source_vectorized_segments, medoid_vectorized_segments):
    """
    Calculates the distances from every source_document (reprsented by its segments) to every medoid
    document (also represented by its segments) using the hausdorff distance.
    
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param source_vectorized_segments: array of length M, where each element is a matrix with one row
        for every segment in a source document
    :param medoid_vectorized_segments: array of length N where each element is a matrix with one row
        for every segment in a medoid document
    :return: distances to medoids (MxN matrix)
    """
    
    num_test_samples = len(source_vectorized_segments)
    num_medoids = len(medoid_vectorized_segments)
    
    test_dataset = np.zeros((num_test_samples,num_medoids))    
    
    for i,source_segments in enumerate(source_vectorized_segments):
        for j,medoid_segments in enumerate(medoid_vectorized_segments):
            test_dataset[i][j] = hausdorff(source_segments.toarray(),medoid_segments.toarray())
            
    return np.array(test_dataset)

In [ ]:
# segments, documents and labelsets are defined outside of the parameterGrid loop
# because they're the same for every configuration    
segments_train, segments_test, documents_train, documents_test, Y_train, Y_test = train_test_split(segments,
                                                                                               documents,
                                                                                               labels,
                                                                                               test_size=0.15)

print('total number of train documents: {}'.format(len(documents_train)))
print('total number of validation documents: {}'.format(len(documents_test)))

In [ ]:
# the binarizer needs to be fit on all labels
mlb = MultiLabelBinarizer()
mlb.fit(labels)

Y_train = mlb.transform(Y_train)
Y_test = mlb.transform(Y_test)

print("total number of unique tags: {} ".format(len(mlb.classes_)))

### these were found by grid search

In [ ]:
final_parameters = [
    {
        'medoid_normalization':  [None],
        'svm_kernel': ['poly'],
        'svm_c':[1.0],
        'svm_degree' :[3],
        'svm_gamma':['auto'],
        'vectorizer_norm': [None],
        'nb_medoids_ratio': [0.2],
        'max_features':[500]
    }
]

In [ ]:
for (i,configuration) in enumerate(ParameterGrid(final_parameters)):
    
    tfidf_vectorizer = CountVectorizer(
        max_features=configuration['max_features'], 
        norm=configuration['vectorizer_norm'])
    
    # TRAINING SET
    tfidf_vectorizer.fit(documents_train)
    tfidf_segments_train = vectorize_segments(segments_train, tfidf_vectorizer)
        
    # THE FOLLOWING BLOCK TAKES SOME TIME, BUT IT WILL ONLY RUN ONCE
    
    path_to_cache = MODELS_ROOT.rstrip('/') + "/distance-matrix-train-{}-{}-{}.p".format(
        configuration['max_features'],
        configuration['vectorizer_norm'],
        SAMPLE_FRAC)
    
    if os.path.isfile(path_to_cache):
        print('cache hit')
        dist_matrix_train = pickle.load(open(path_to_cache,"rb"))
    else:
        print('Fitting distance matrix for norm={}'.format(configuration['vectorizer_norm']))
        
        dist_matrix_train = make_distance_matrix_for_segments(tfidf_segments_train)
        pickle.dump(dist_matrix_train, open(path_to_cache, "wb"))
    
    # nb_medoids depends upon the dataset length
    ratio = configuration['nb_medoids_ratio']
    nb_medoids = int(len(tfidf_segments_train) * ratio)
    
    medoids_indices_train = k_medoids(dist_matrix_train,nb_medoids)[0]

    X_train = make_train_dataset(dist_matrix_train,medoids_indices_train)
    
    # TEST SET
    
    tfidf_segments_test = vectorize_segments(segments_test, tfidf_vectorizer)
          
    # medoids trained on the training set
    fitted_medoids = tfidf_segments_train[medoids_indices_train]
    X_test = make_test_dataset(tfidf_segments_test,fitted_medoids)     
        
    svm = SVC(kernel=configuration['svm_kernel'],
            gamma=configuration['svm_gamma'],
            C=configuration['svm_c'],
            degree=configuration['svm_degree'])
    
    clf = OneVsRestClassifier(CalibratedClassifierCV(svm,cv=2),n_jobs=-1)        

    if configuration['medoid_normalization'] == 'standard':      
        scaler = StandardScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    elif configuration['medoid_normalization'] == 'minmax':
        scaler = MinMaxScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    else:
        X_train_final = X_train
        X_test_final = X_test
    
    # y_train was defined outside the loop    
    clf.fit(X_train,Y_train)
    
    # train score
    Y_pred_train = clf.predict_proba(X_train)
    
    # validation score
    Y_pred_test = clf.predict_proba(X_test)  
    
    print("iter: {}, configuration: {}\n".format(i,configuration))
    
    ks = [1,2,3,4,5,6,7,8,9,10]

    for k in ks:
        print("train micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k,normalize=True)))
        print("validation micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_test,Y_pred_test,k=k,normalize=True)))    

In [ ]:
plt.clf()
img = plt.gcf()
ax = plt.gca()
validation_scores = [

]
plot_micro_f1_at_k(validation_scores,ax)
plt.gcf().set_size_inches(7,5)
plt.gca().legend_.remove()
plt.show()